<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>
<h1 align=right><font size = 4>by Pavel Milchev</font></h1>

<h1> Step 1 - Parse data from site and shape it</h1>

*Install the needed librabies.*

In [48]:
!pip install beautifulsoup4 # used to parse the html
print("BeautifulSoup 4 is installed!")
!pip install html5lib # already installed but was in the example ^_^
print("HTML5 libraty is installed!")
!pip install lxml # already installed but was in the example ^_^
print("lxml libraty is installed!")

BeautifulSoup 4 is installed!
HTML5 libraty is installed!
lxml libraty is installed!


*Import the needed libraries*

BeautifulSoup is used by pd.read_html

In [49]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd

*Take the complete text of the wiki-page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and create a data frame*

**I assume that:** I need the first table from the page and that there is at least one

In [50]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

**I assume that:** the names of the columns from the html page are as follows: Postcode, Borough, Neighbourhood

In [51]:
# rename the column to ber the same as the expected from the assignement
df.rename(columns={'Postcode':'PostalCode'}, inplace=True)

*Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.*

In [52]:
#Ignore cells with a borough that is Not assigned.
indexNames = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNames , inplace=True)

*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.*

In [53]:
nonNeighbourhoods = df[df['Neighbourhood'] == 'Not assigned'].index
for k in nonNeighbourhoods:
    df.at[k,'Neighbourhood'] = df.at[k,'Borough']

***The procedure to fulfill the following requirement:*** More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [64]:
# create a new empty DataFrame to store the combined Neighbourhoods
refined_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])
# group by PostalCode to find the Neighbourhoods with the same postal
grouped = df.groupby(['PostalCode'], sort=False)
for postalCode, postalCode_df in grouped:
    # transform the list of unique neighbourhoods to a comma separated single string
    postalCode_df['Neighbourhood'] = ",".join(postalCode_df['Neighbourhood'].unique())
    # drop duplicated rows based on the Borough column
    postalCode_df = postalCode_df.drop_duplicates(subset='Borough')
    # add the current dataframe with ONE LINE to the final refined_df
    refined_df = pd.concat([refined_df, postalCode_df])
    
# reindex the table starting the rows from 0
refined_df.reset_index( drop=True, inplace = True)
print(refined_df.shape)
refined_df

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


(103, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


In [65]:
#save the transformed data frame for the next steps of the assignement
refined_df.to_csv('toronto_postal_step1.csv', mode='w')


<h1> Step 2 - Add geolocaion to the data </h1>